In [1]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis_lvm import run_ibis_lvm

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [2]:
existing_directory = None
task_handle = 'ez_lvm'
gen_model = 0

# if existing_directory is None:
#     log_dir = make_folder(task_handle)  
#     print("\n\nCreating new directory: %s" % log_dir)

# else:
#     log_dir = existing_directory
#     log_dir = path_backslash(log_dir)
#     print("\n\nReading from existing directory: %s" % log_dir)

log_dir = './log/20210101_210037_ez_lvm/'

## Create Data

In [3]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 1, 
    size = 200,
    random_seed = 6
    )
    
exp_data.generate()
save_obj(exp_data, 'data', log_dir)

# exp_data = load_obj('data', log_dir)

## Run HMC

In [4]:
# sm = load_obj('sm', 'log/compiled_models/model8/')

import pystan

with open(
    'codebase/stancode/models/1factor/model_1.stan',
    'r'
    ) as file:
    model_code = file.read()

    sm = pystan.StanModel(model_code=model_code, verbose=False)
save_obj(sm, 'sm_hmc', log_dir)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7b691d096ece9442c80a4dc5e87eb167 NOW.


In [5]:
# sm = load_obj('sm', 'log/compiled_models/model8/')
# sm = load_obj('sm_hmc', log_dir)

fit_run = run_stan_model(
    data = exp_data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
#     initial_values={'beta':[0,0,0,0,0,0]},
    adapt_engaged=True,
    )

In [7]:
param_names = ['beta', 'alpha', 'betabeta']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)

In [4]:
ps = load_obj( 'ps_hmc', log_dir)

In [5]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
plot_density(df) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

## Run IBIS

In [12]:
# load existing results

# log_dir = 'log/20201218_172340_ez_comp/'
# particles = load_obj('particles', log_dir)

In [6]:
gen_model = False
model_num = 7
degeneracy_limit = 0.5
gen_model = False


ibis = run_ibis_lvm(
    exp_data,
    model_num,
    1000,
    100,
    gen_model,
    log_dir
    )


100%|██████████| 200/200 [11:35:01<00:00, 208.51s/it]   




Marginal Likelihood 9.32793


## Plot Both results

In [ ]:
# have to resample particles to get rid of weights
# particles = ibis['particles']
particles.resample_particles()

In [ ]:
np.mean(np.squeeze(particles.particles['beta']),0)

In [ ]:
param = 'beta'
df = get_post_df(ps[param].reshape(1000,1,6,1)) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

In [ ]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 